In [1]:
pattern = '*1'

In [2]:
import xmltodict as xd
import pandas as pd
import os, glob
import mio
import numpy as np
import geopandas as gpd
from shapely.geometry import Point, Polygon
import pathlib
import time

In [3]:
def parse_model(dae_file:str) -> dict:
    " get point and vertex list for one model"
    with open(dae_file) as fin:
        s = fin.read()
    d = xd.parse(s)
    
    geometries = d['COLLADA']['library_geometries']['geometry']
    
    res = {}
    for geo in geometries:
        geo_id = geo['@id']

        # float list
        f_list = geo['mesh']['source']['float_array']['#text'].split()
        f_list = [float(f) for f in f_list]

        # vertex list
        v_list = geo['mesh']['triangles']['p'].split()
        v_list = [int(n) for n in v_list]
        # write result
        
        res[geo_id] = (f_list, v_list)
    
    return res

In [4]:
def comb_f_v(f_list, v_list):
    va = np.array(v_list)
    va = va.reshape(len(v_list)//3, 3)
    v = pd.DataFrame(va, columns=['v0', 'v1', 'v2'])
    
    fa = np.array(f_list)
    fa = fa.reshape(len(f_list)//3, 3)
    f = pd.DataFrame(fa, columns=['x', 'y', 'z'])
    
    assert max(v.v0.max(), v.v1.max(), v.v2.max()) +1 == len(f)
    
    # combine the lists
    res = (
        v.join(f, on='v0').rename(columns={'x':'x0', 'y':'y0', 'z':'z0'})
         .join(f, on='v1').rename(columns={'x':'x1', 'y':'y1', 'z':'z1'})
         .join(f, on='v2').rename(columns={'x':'x2', 'y':'y2', 'z':'z2'})
         .drop(columns=['v0', 'v1', 'v2'])
    )
    
    return res

In [5]:
def dae2df(dae_path):
    dae_path = pathlib.Path(dae_path)
    dic = parse_model(dae_path)
    ldf = []
    for key in dic:
        f_list, v_list = dic[key]
        df = comb_f_v(f_list, v_list)
        ldf.append(df)
    
    res = pd.concat(ldf)
    res.insert(0, 'model', dae_path.name)
    return res

In [6]:
def get_model_list(kml_path:str)->pd.DataFrame:
    " read kml file, get path and anchor coordinates for every model"
    with open(kml_path) as fin:
        s = fin.read()
    kmld =  xd.parse(s)
    rows = []
    for pm in kmld['kml']['Document']['Placemark'][0:]:
        model = pm['Model']
        row = {}
        row['model'] = os.path.basename(model['Link']['href'])
        sLon = model['Location']['longitude']
        sLat = model['Location']['latitude']
        row['swiss_lon'] = float(sLon)
        row['swiss_lat'] = float(sLat)
        row['swiss_x'], row['swiss_y'] = mio.wgs_swiss(sLon, sLat)
        rows.append(row)
    df = pd.DataFrame(rows)
    
    mdir = str(pathlib.Path(kml_path).parent / 'models')
    res = (
        df.assign(model_path = lambda d : mdir + '\\' + d.model)
        .set_index('model')
    )
    return res

In [7]:
# create datafrme with all triangles
#
def make_tri_tab(kml):
    p_kml = pathlib.Path(kml)
    mod_list = get_model_list(p_kml)
    ldf = []
    i = 0
    for ind, row in mod_list.iterrows():
        i += 1
        mio.show_perc(i, len(mod_list), 100)
        try:
            tris = dae2df(row.model_path)
            tris.insert(0, 'kml', p_kml.name)
            tris.insert(2, 'swiss_x', row.swiss_x)
            tris.insert(3, 'swiss_y', row.swiss_y)
            ldf.append(tris)
        except:
            print(f'Problem with {row.model_path}')
    df = pd.concat(ldf)

    # create a geo dataframe
    vec = (
        pd.DataFrame()
        .assign(kml = df.kml)
        .assign(model = df.model)
        .assign(x0 = df.x0 + df.swiss_x, y0 = df.y0 + df.swiss_y, z0 = df.z0)
        .assign(x1 = df.x1 + df.swiss_x, y1 = df.y1 + df.swiss_y, z1 = df.z1)
        .assign(x2 = df.x2 + df.swiss_x, y2 = df.y2 + df.swiss_y, z2 = df.z2)
    )
    geom = [Polygon( [(t[0], t[1]), (t[2], t[3]), (t[4], t[5])]) for t in zip(vec.x0, vec.y0, vec.x1, vec.y1, vec.x2, vec.y2)]
    gdf = gpd.GeoDataFrame(vec, geometry=geom)
    # filter out vertical walls
    res = gdf[gdf.geometry.area > 0]
    return gdf

In [8]:
def get_tag_todo():
    source_dir = pathlib.Path("../sb_KML")
    tags_all = set([d.stem for d in source_dir.glob(pattern)])

    dest_dir = pathlib.Path('../sb_tab')
    tags_done =  set([t.stem for t in dest_dir.glob('*.tab')])

    tags_todo = tags_all.difference(tags_done)
    l = list(tags_todo)
    l.sort()
    
    if len(l) > 0:
        print(f"{len(l)} files to do")
        return l[0]
    else:
        return

In [9]:
get_tag_todo()

796 files to do


'1031-31'

In [ ]:
# MAIN
while(True):
    tag = get_tag_todo()
    if tag is not None:
        print(f'processing {tag}')
        kml = f"../sb_KML/{tag}/{tag}.kml"
        gdf = make_tri_tab(kml)
        print(f'writing {tag}.tab')
        mio.write_tab(gdf, f'../sb_tab/{tag}.tab')
    time.sleep(10)

796 files to do
processing 1031-31
5.91% 11.82% 17.73% 23.64% 29.55% 35.46% 41.37% 47.28% 53.19% 59.1% 65.01% 70.92% 76.83% 82.74% 88.65% 94.56% writing 1031-31.tab
96378 rows of type MultiPolygon written to mapinfo file.
795 files to do
processing 1031-41
5.84% 11.69% 17.53% 23.38% 29.22% 35.07% 40.91% Problem with ..\sb_KML\1031-41\models\model_731.dae
46.76% 52.6% 58.45% 64.29% 70.13% 75.98% 81.82% 87.67% 93.51% 99.36% writing 1031-41.tab
98921 rows of type MultiPolygon written to mapinfo file.
794 files to do
processing 1032-11
8.1% 16.19% 24.29% 32.39% 40.49% 48.58% 56.68% 64.78% 72.87% 80.97% 89.07% 97.17% writing 1032-11.tab
56856 rows of type MultiPolygon written to mapinfo file.
793 files to do
processing 1032-21
writing 1032-21.tab
5392 rows of type MultiPolygon written to mapinfo file.
792 files to do
processing 1032-31
2.33% 4.65% 6.98% 9.31% 11.64% 13.96% 16.29% 18.62% 20.94% 23.27% 25.6% 27.93% 30.25% 32.58% 34.91% 37.24% 39.56% 41.89% 44.22% 46.54% 48.87% 51.2% 53.53% 55